# Evolving Rubrics (DR-Tulu)

In [9]:
import pandas as pd

df = pd.read_csv('data/moose-chem/chem_research_2024-sample.csv')
df.head()

,No,Title,Public Date,Publisher,Background Little Survey,Background Little Survey (strict),Background Question,Background Question (strict),Main Inspiration,Inspiration paper 1 title,Relation between the main inspiration and the inspiration paper 1,Inspiration paper 2 title,Relation between the main inspiration and the inspiration paper 2,Inspiration paper 3 title,Relation between the main inspiration and the inspiration paper 3,Main hypothesis,Experiments to Verify the Research Hypothesis:,Reasoning Process,Note
0,0,"Ultrastrong, flexible thermogalvanic armor wit...",2024/08/07,Nature Communication,Background Survey Summary\nPrior to the develo...,Background Survey Summary\nPrior to the develo...,How can we design a flexible thermogalvanic de...,NaN,The main inspiration for the paper stems from ...,Interactions between macromolecules and ions: ...,Their work on the Hofmeister series (cited as ...,Beyond Hofmeister,Insights into using the Hofmeister series to m...,Strong tough hydrogels via the synergy of free...,This paper demonstrates the synergy of freeze-...,By integrating guanidine sulfate (Gdm)2SO4 int...,Thermoelectric Performance Tests: The FTGA exh...,bkg + insp1/insp2 + insp3 = hyp,insp1/2: The Hofmeister series; insp3: directi...
1,1,Chaotropic Effect-Boosted Thermogalvanic Ionog...,2024/01/29,Advanced Materials,Thermoelectric Energy Conversion: Traditional ...,Thermoelectric Energy Conversion: Traditional ...,How can a thermoelectric system be designed to...,How can a thermoelectric system be designed to...,The main inspiration of the paper comes from t...,The Chaotropic Effect as an Assembly Motif in ...,"Have the keyword ""Chaotropic Effect""",Role of Ions in Hydrogels with an Ionic Seebec...,This paper explains how the Soret effect and s...,Hierarchically porous polymer coatings for hig...,This paper details the creation of radiative c...,The hypothesis of the paper is that integratin...,The experiments included the design and testin...,bkg + insp1 + insp2 + insp3 = hyp,insp1: Chaotropic Effect (ref id: find online ...
2,2,Synergistic Anisotropic Network and Hierarchic...,2024/02/01,Small,Background survey:\n\nHarnessing Low-Grade Hea...,NaN,How can a cost-effective N-type quasi-solid-st...,NaN,The main inspiration of this work is the syner...,3D Hierarchical Electrodes Boosting Ultrahigh ...,It directly inspires the design of the hierarc...,Cu(ii/i) redox couples: potential alternatives...,This work supports the use of copper as the re...,Anti-Fatigue and Highly Conductive Thermocells...,It directly inspired the structural design of ...,The hypothesis is that combining an anisotropi...,The experiments designed to test the hypothesi...,bkg + insp1 + insp2 + insp3 = hyp,insp1: Hierarchical 3D Copper Electrodes (find...
3,3,Liquid-flow thermocells with high hybrid entro...,2024/07/14,Nano Energy,Background Survey\nThermoelectric Conversion f...,NaN,How can the thermoelectric performance of liqu...,How can the thermoelectric performance of liqu...,The main inspiration of this paper is the hybr...,Effect of solvation shell structure on thermop...,It demonstrated theThis paper serves as a foun...,High thermopower of ferri/ferrocyanide redox c...,This paper introduces the concept of using a c...,NaN,NaN,The hypothesis is that inducing hybrid entropy...,The experiments include fabricating LFTCs with...,bkg + insp1 + insp2 = hyp,insp1: Hybrid Entropy Increase through Flow-Se...
4,4,Tough and elastic hydrogel thermocells for hea...,2024/08/01,Chemical Engineering Journal,Background Survey:\nHydrogel thermocells have ...,NaN,How can a hydrogel thermocell be designed to a...,How can a hydrogel thermocell be designed to a...,The main inspiration behind this paper is a bi...,"Fracture, fatigue, and friction of polymers in...",This paper focuses on the role of dense polyme...,Highly stretchable and tough hydrogels,"This paper shows how a dual-network approach, ...",A review of radiation-grafted polymer electrol...,increased m

In [10]:
row_n = 0
row = df.loc[row_n]
background_survey = df.loc[row_n, 'Background Little Survey']
background_question = df.loc[row_n, 'Background Question']

print(f"Row: {row_n}")
print(f"Title: {df.loc[row_n, 'Title']}")
print(f"Public Date: {df.loc[row_n, 'Public Date']}")
print(f"Publisher: {df.loc[row_n, 'Publisher']}\n")
print(f"Background Little Survey: \n{df.loc[row_n, 'Background Little Survey']}\n")
print(f"Background Question: \n{df.loc[row_n, 'Background Question']}")

Row: 0
Title: Ultrastrong, flexible thermogalvanic armor with a Carnot-relative efficiency over 8%
Public Date: 2024/08/07
Publisher: Nature Communication

Background Little Survey: 
Background Survey Summary
Prior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:
Thermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to electricity, often faced limitations in efficiency, especially when dealing with low-grade heat sources like body heat. These devices typically suffered from low thermopower and were limited by rigid structures, which made them less suitable for wearable applications.
2. Quasi-Solid Thermocells: The emergence of quasi-solid thermocells, which used gel networks to confine liquid electrolytes, provided a 

In [11]:
question = f"""A continuación se muestra un resumen de contexto de un trabajo científico, seguido de una pregunta que debe responderse usando únicamente la información contenida en dicho resumen.

---
{background_survey}

Pregunta:
{background_question}

Tarea: Utilizando únicamente el resumen de contexto proporcionado, redacta una respuesta clara y completa a la pregunta indicada.
"""

print(question)

A continuación se muestra un resumen de contexto de un trabajo científico, seguido de una pregunta que debe responderse usando únicamente la información contenida en dicho resumen.

---
Background Survey Summary
Prior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:
Thermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to electricity, often faced limitations in efficiency, especially when dealing with low-grade heat sources like body heat. These devices typically suffered from low thermopower and were limited by rigid structures, which made them less suitable for wearable applications.
2. Quasi-Solid Thermocells: The emergence of quasi-solid thermocells, which used gel networks to confine liquid electrolytes, provided

In [ ]:
from evolving_rubrics import evolve_rubrics_for_example

# Ejecutar evolución con 3 iteraciones para el mismo ejemplo
result = await evolve_rubrics_for_example(
    question=question,
    num_iterations=3,  # Itera 3 veces para el mismo ejemplo
    num_responses_per_iteration=4  # Genera 4 respuestas por iteración
)

# Acceder a los resultados
print(result['initial_ground_truth'])
print(result['iterations'])  # Lista con resultados de cada iteración
print(result['final_ground_truth'])

# DEV Evolving Rubrics (DR-Tulu)

## Evolucion test

In [1]:
import json
import os
import re
import asyncio
from typing import List, Dict, Any, Optional

# Cargar variables de entorno desde .env si existe
try:
    from dotenv import load_dotenv
    load_dotenv()  # Carga .env automáticamente
except ImportError:
    # Si no está instalado dotenv, simplemente no carga el .env
    pass

# Solo necesitamos OpenAI (o puedes usar Anthropic, etc.)
try:
    from openai import AsyncOpenAI, AsyncAzureOpenAI
except ImportError:
    print("Error: Necesitas instalar openai: pip install openai")
    exit(1)

In [2]:
# from drtulu_rubric_evolve_simple import evaluar_respuesta_completa, evaluar_rubrica, generar_rubricas_adaptativas, actualizar_ground_truth

In [ ]:
# ============================================================================
# CONFIGURACIÓN
# ============================================================================

from dotenv import load_dotenv
load_dotenv()  # Esto carga variables desde un archivo .env si existe

# Configuración para Azure OpenAI usando variables de entorno (posiblemente del .env)
USE_AZURE = os.environ.get("USE_AZURE_OPENAI", "false").lower() == "true"
AZURE_API_BASE = os.environ.get("AZURE_API_BASE", "https://development-cursor-models.openai.azure.com/")
AZURE_API_KEY = os.environ.get("AZURE_API_KEY", "")
AZURE_API_VERSION = os.environ.get("AZURE_API_VERSION", "2024-02-15-preview")

USE_AZURE: True
AZURE_API_BASE: https://development-cursor-models.openai.azure.com/
AZURE_API_KEY: Set
AZURE_API_VERSION: 2024-12-01-preview


In [4]:
# Modelos (si usas Azure, estos son los nombres de los deployments)
RUBRIC_GENERATION_MODEL = os.environ.get("RUBRIC_GENERATION_MODEL", "gpt-4o-mini")
RUBRIC_JUDGE_MODEL = os.environ.get("RUBRIC_JUDGE_MODEL", "gpt-4o-mini")

# Inicializar cliente OpenAI o Azure OpenAI
if USE_AZURE:
    if not AZURE_API_KEY:
        print("⚠️  Advertencia: USE_AZURE_OPENAI=true pero AZURE_API_KEY no está configurada")
    client = AsyncAzureOpenAI(
        api_version=AZURE_API_VERSION,
        azure_endpoint=AZURE_API_BASE.rstrip('/'),
        api_key=AZURE_API_KEY or os.environ.get("OPENAI_API_KEY", "")
    )
    print(f"✓ Usando Azure OpenAI")
    print(f"  Endpoint: {AZURE_API_BASE}")
    print(f"  API Version: {AZURE_API_VERSION}")
    print(f"  Modelo generación: {RUBRIC_GENERATION_MODEL}")
    print(f"  Modelo juez: {RUBRIC_JUDGE_MODEL}")
else:
    client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    print(f"✓ Usando OpenAI estándar")

✓ Usando Azure OpenAI
  Endpoint: https://development-cursor-models.openai.azure.com/
  API Version: 2024-12-01-preview
  Modelo generación: gpt-5-mini
  Modelo juez: gpt-5-mini


In [24]:
# ============================================================================
# FUNCIONES AUXILIARES (replicadas del proyecto)
# ============================================================================

def extract_json_from_response(response: str) -> Optional[Dict]:
    """Extrae un objeto JSON de una respuesta de texto."""
    # Buscar JSON en bloques de código
    json_match = re.search(r'```json\s*(\{.*?\})\s*```', response, re.DOTALL)
    if json_match:
        try:
            return json.loads(json_match.group(1))
        except json.JSONDecodeError:
            pass
    
    # Buscar JSON directo
    json_match = re.search(r'\{.*\}', response, re.DOTALL)
    if json_match:
        try:
            return json.loads(json_match.group(0))
        except json.JSONDecodeError:
            pass
    
    return None


async def llamar_llm(prompt: str, system_prompt: Optional[str] = None, model: str = "gpt-4o-mini") -> str:
    """Llama a un LLM de forma asíncrona (soporta OpenAI y Azure OpenAI)."""
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    # Configurar parámetros según si es Azure o OpenAI estándar
    if USE_AZURE:
        # Para Azure OpenAI, usamos AsyncAzureOpenAI con la configuración correcta
        azure_client = AsyncAzureOpenAI(
            api_version=AZURE_API_VERSION,
            azure_endpoint=AZURE_API_BASE.rstrip('/'),
            api_key=AZURE_API_KEY or os.environ.get("OPENAI_API_KEY", "")
        )
        
        # En Azure, pasamos el modelo (deployment name) en el create()
        # Azure OpenAI usa max_completion_tokens en lugar de max_tokens
        # Este modelo no acepta temperature=0, solo el valor por defecto (1)
        response = await azure_client.chat.completions.create(
            model=model,
            messages=messages,
            max_completion_tokens=4000
        )
    else:
        # OpenAI estándar
        response = await client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0,
            max_tokens=4000
        )
    
    return response.choices[0].message.content

### Dataset original

In [ ]:
# # TOY EXAMPLE

# # ========================================================================
# # PASO 1: Dataset Original
# # ========================================================================
# print("\n📋 PASO 1: Dataset Original")
# print("-" * 70)

# pregunta = "¿Qué es la inteligencia artificial?"


📋 PASO 1: Dataset Original
----------------------------------------------------------------------
Pregunta: ¿Qué es la inteligencia artificial?

Rubricas Persistentes: 2
  1. [Definición Clara] La respuesta debe definir claramente qué es la inteligencia artificial...
  2. [Ejemplos Prácticos] La respuesta debe incluir ejemplos prácticos de aplicaciones de IA...


In [ ]:
import pandas as pd

df = pd.read_csv('data/moose-chem/chem_research_2024-sample.csv')
df.head()

,No,Title,Public Date,Publisher,Background Little Survey,Background Little Survey (strict),Background Question,Background Question (strict),Main Inspiration,Inspiration paper 1 title,Relation between the main inspiration and the inspiration paper 1,Inspiration paper 2 title,Relation between the main inspiration and the inspiration paper 2,Inspiration paper 3 title,Relation between the main inspiration and the inspiration paper 3,Main hypothesis,Experiments to Verify the Research Hypothesis:,Reasoning Process,Note
0,0,"Ultrastrong, flexible thermogalvanic armor wit...",2024/08/07,Nature Communication,Background Survey Summary\nPrior to the develo...,Background Survey Summary\nPrior to the develo...,How can we design a flexible thermogalvanic de...,NaN,The main inspiration for the paper stems from ...,Interactions between macromolecules and ions: ...,Their work on the Hofmeister series (cited as ...,Beyond Hofmeister,Insights into using the Hofmeister series to m...,Strong tough hydrogels via the synergy of free...,This paper demonstrates the synergy of freeze-...,By integrating guanidine sulfate (Gdm)2SO4 int...,Thermoelectric Performance Tests: The FTGA exh...,bkg + insp1/insp2 + insp3 = hyp,insp1/2: The Hofmeister series; insp3: directi...
1,1,Chaotropic Effect-Boosted Thermogalvanic Ionog...,2024/01/29,Advanced Materials,Thermoelectric Energy Conversion: Traditional ...,Thermoelectric Energy Conversion: Traditional ...,How can a thermoelectric system be designed to...,How can a thermoelectric system be designed to...,The main inspiration of the paper comes from t...,The Chaotropic Effect as an Assembly Motif in ...,"Have the keyword ""Chaotropic Effect""",Role of Ions in Hydrogels with an Ionic Seebec...,This paper explains how the Soret effect and s...,Hierarchically porous polymer coatings for hig...,This paper details the creation of radiative c...,The hypothesis of the paper is that integratin...,The experiments included the design and testin...,bkg + insp1 + insp2 + insp3 = hyp,insp1: Chaotropic Effect (ref id: find online ...
2,2,Synergistic Anisotropic Network and Hierarchic...,2024/02/01,Small,Background survey:\n\nHarnessing Low-Grade Hea...,NaN,How can a cost-effective N-type quasi-solid-st...,NaN,The main inspiration of this work is the syner...,3D Hierarchical Electrodes Boosting Ultrahigh ...,It directly inspires the design of the hierarc...,Cu(ii/i) redox couples: potential alternatives...,This work supports the use of copper as the re...,Anti-Fatigue and Highly Conductive Thermocells...,It directly inspired the structural design of ...,The hypothesis is that combining an anisotropi...,The experiments designed to test the hypothesi...,bkg + insp1 + insp2 + insp3 = hyp,insp1: Hierarchical 3D Copper Electrodes (find...
3,3,Liquid-flow thermocells with high hybrid entro...,2024/07/14,Nano Energy,Background Survey\nThermoelectric Conversion f...,NaN,How can the thermoelectric performance of liqu...,How can the thermoelectric performance of liqu...,The main inspiration of this paper is the hybr...,Effect of solvation shell structure on thermop...,It demonstrated theThis paper serves as a foun...,High thermopower of ferri/ferrocyanide redox c...,This paper introduces the concept of using a c...,NaN,NaN,The hypothesis is that inducing hybrid entropy...,The experiments include fabricating LFTCs with...,bkg + insp1 + insp2 = hyp,insp1: Hybrid Entropy Increase through Flow-Se...
4,4,Tough and elastic hydrogel thermocells for hea...,2024/08/01,Chemical Engineering Journal,Background Survey:\nHydrogel thermocells have ...,NaN,How can a hydrogel thermocell be designed to a...,How can a hydrogel thermocell be designed to a...,The main inspiration behind this paper is a bi...,"Fracture, fatigue, and friction of polymers in...",This paper focuses on the role of dense polyme...,Highly stretchable and tough hydrogels,"This paper shows how a dual-network approach, ...",A review of radiation-grafted polymer electrol...,increased m

In [ ]:
row_n = 0
row = df.loc[row_n]
background_survey = df.loc[row_n, 'Background Little Survey']
background_question = df.loc[row_n, 'Background Question']

print(f"Row: {row_n}")
print(f"Title: {df.loc[row_n, 'Title']}")
print(f"Public Date: {df.loc[row_n, 'Public Date']}")
print(f"Publisher: {df.loc[row_n, 'Publisher']}\n")
print(f"Background Little Survey: \n{df.loc[row_n, 'Background Little Survey']}\n")
print(f"Background Question: \n{df.loc[row_n, 'Background Question']}")

Row: 0
Title: Ultrastrong, flexible thermogalvanic armor with a Carnot-relative efficiency over 8%
Public Date: 2024/08/07
Publisher: Nature Communication

Background Little Survey: 
Background Survey Summary
Prior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:
Thermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to electricity, often faced limitations in efficiency, especially when dealing with low-grade heat sources like body heat. These devices typically suffered from low thermopower and were limited by rigid structures, which made them less suitable for wearable applications.
2. Quasi-Solid Thermocells: The emergence of quasi-solid thermocells, which used gel networks to confine liquid electrolytes, provided a 

In [17]:
question = f"""A continuación se muestra un resumen de contexto de un trabajo científico, seguido de una pregunta que debe responderse usando únicamente la información contenida en dicho resumen.

---
{background_survey}

Pregunta:
{background_question}

Tarea: Utilizando únicamente el resumen de contexto proporcionado, redacta una respuesta clara y completa a la pregunta indicada.
"""

print(question)

A continuación se muestra un resumen de contexto de un trabajo científico, seguido de una pregunta que debe responderse usando únicamente la información contenida en dicho resumen.

---
Background Survey Summary
Prior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:
Thermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to electricity, often faced limitations in efficiency, especially when dealing with low-grade heat sources like body heat. These devices typically suffered from low thermopower and were limited by rigid structures, which made them less suitable for wearable applications.
2. Quasi-Solid Thermocells: The emergence of quasi-solid thermocells, which used gel networks to confine liquid electrolytes, provided

### Generacion rubricas originales

In [ ]:
# # TOY EXAMPLE

# ground_truth_inicial = {
#     "query": pregunta,
#     "rubrics": [
#         {
#             "description": "La respuesta debe definir claramente qué es la inteligencia artificial",
#             "weight": 1.0,
#             "title": "Definición Clara"
#         },
#         {
#             "description": "La respuesta debe incluir ejemplos prácticos de aplicaciones de IA",
#             "weight": 1.0,
#             "title": "Ejemplos Prácticos"
#         }
#     ]
# }

# print(f"Pregunta: {pregunta}")
# print(f"\nRubricas Persistentes: {len(ground_truth_inicial['rubrics'])}")
# for i, rubric in enumerate(ground_truth_inicial["rubrics"], 1):
#     print(f"  {i}. [{rubric['title']}] {rubric['description']}...")

In [ ]:
async def generar_rubricas_originales(pregunta: str, model: str = None) -> Dict[str, Any]:
    if model is None:
        model = RUBRIC_GENERATION_MODEL
    
    prompt = f"""Eres un experto en evaluación educativa. Genera rubricas de evaluación para la siguiente pregunta.

Pregunta: {pregunta}

Genera 2-4 rubricas que cubran los aspectos esenciales para evaluar una respuesta a esta pregunta. 
Cada rubrica debe tener:
- title: Un título corto y descriptivo
- description: Una descripción detallada de qué se evalúa
- weight: Un peso (usa 1.0 para todas)

Responde SOLO con un JSON válido en este formato:
{{
  "query": "{pregunta}",
  "rubrics": [
    {{
      "title": "Título de la rubrica",
      "description": "Descripción detallada de qué se evalúa",
      "weight": 1.0
    }}
  ]
}}"""

    try:
        respuesta_llm = await llamar_llm(
            prompt=prompt,
            model=model
        )
        
        rubricas = extract_json_from_response(respuesta_llm)
        
        if rubricas and "rubrics" in rubricas:
            return rubricas
        else:
            print(f"⚠️  No se pudo extraer rubricas válidas")
            print(f"Respuesta recibida: {respuesta_llm[:300]}...")
            # Retornar estructura básica si falla
            return {
                "query": pregunta,
                "rubrics": [
                    {
                        "title": "Respuesta relevante",
                        "description": "La respuesta debe ser relevante a la pregunta",
                        "weight": 1.0
                    }
                ]
            }
    except Exception as e:
        print(f"❌ Error generando rubricas originales: {e}")
        return {
            "query": pregunta,
            "rubrics": [
                {
                    "title": "Respuesta relevante",
                    "description": "La respuesta debe ser relevante a la pregunta",
                    "weight": 1.0
                }
            ]
        }


In [ ]:
# Paso 2: Generar rubricas originales usando LLM
print("\n📋 PASO 1: Generando rubricas originales con LLM...")
print("-" * 70)

ground_truth_inicial = await generar_rubricas_originales(question)

print(f"✓ Rubricas generadas: {len(ground_truth_inicial['rubrics'])}")
for i, rubric in enumerate(ground_truth_inicial["rubrics"], 1):
    print(f"  {i}. [{rubric['title']}] {rubric['description'][:80]}...")



📋 PASO 1: Generando rubricas originales con LLM...
----------------------------------------------------------------------
✓ Rubricas generadas: 4
  1. [Adherencia al resumen] Evalúa si la respuesta se basa exclusivamente en la información proporcionada en...
  2. [Análisis del compromiso eficiencia–robustez] Mide la calidad del análisis sobre el trade-off entre eficiencia termogalvánica ...
  3. [Propuesta de diseño coherente con objetivos] Valora si la respuesta propone una solución de diseño que busque simultáneamente...
  4. [Claridad y lógica de la argumentación] Evalúa la claridad expositiva, la organización y la lógica interna de la respues...


In [29]:
ground_truth_inicial

{'query': "A continuación se muestra un resumen de contexto de un trabajo científico, seguido de una pregunta que debe responderse usando únicamente la información contenida en dicho resumen.\n\n---\nBackground Survey Summary\nPrior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:\nThermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to electricity, often faced limitations in efficiency, especially when dealing with low-grade heat sources like body heat. These devices typically suffered from low thermopower and were limited by rigid structures, which made them less suitable for wearable applications.\n2. Quasi-Solid Thermocells: The emergence of quasi-solid thermocells, which used gel networks to confine liquid elect

### Respuestas del modelo

In [ ]:
# # TOY EXAMPLE

# # ========================================================================
# # PASO 2: Respuestas del modelo
# # ========================================================================
# print("\n\n🤖 PASO 2: Respuestas generadas por el modelo")
# print("-" * 70)

# respuestas = [
#     "La inteligencia artificial (IA) es la capacidad de las máquinas de realizar tareas que normalmente requieren inteligencia humana, como el reconocimiento de patrones y el aprendizaje. Ejemplos incluyen asistentes virtuales como Siri y sistemas de recomendación.",
#     "La IA es cuando las computadoras hacen cosas inteligentes. Hay muchos ejemplos como robots y chatbots.",
#     "La inteligencia artificial es un campo de la informática que busca crear sistemas capaces de simular funciones cognitivas humanas. Incluye técnicas como machine learning, procesamiento de lenguaje natural y visión por computadora. Aplicaciones prácticas incluyen diagnóstico médico asistido por IA, traducción automática, vehículos autónomos, y sistemas de detección de fraudes financieros. La IA puede anticipar casos límite y manejar situaciones complejas.",
#     "La inteligencia artificial es cuando las máquinas aprenden de datos. Por ejemplo, cuando Netflix te recomienda películas basándose en lo que viste antes, eso es IA. También cuando los coches se conducen solos. La IA causa que las máquinas sean inteligentes porque hay más datos disponibles."
# ]

# print(f"Se generaron {len(respuestas)} respuestas")



🤖 PASO 2: Respuestas generadas por el modelo
----------------------------------------------------------------------
Se generaron 4 respuestas


In [ ]:
async def generar_respuestas_modelo(pregunta: str, num_respuestas: int = 4, model: str = None) -> List[str]:
    if model is None:
        model = RUBRIC_GENERATION_MODEL
    
    respuestas = []
    
    # Generar respuestas con diferentes instrucciones para obtener variedad
    instrucciones_variadas = [
        "Responde de manera completa y detallada, incluyendo ejemplos específicos.",
        "Responde de manera concisa pero informativa.",
        "Responde de manera muy detallada y técnica, incluyendo conceptos avanzados.",
        "Responde de manera simple y directa, adecuada para principiantes.",
        "Responde de manera equilibrada, balanceando profundidad y claridad.",
    ]
    
    for i in range(num_respuestas):
        instruccion = instrucciones_variadas[i % len(instrucciones_variadas)]
        
        prompt = f"""{instruccion}

Pregunta: {pregunta}

Responde la pregunta de manera clara y completa."""

        try:
            respuesta = await llamar_llm(
                prompt=prompt,
                model=model
            )
            respuestas.append(respuesta.strip())
        except Exception as e:
            print(f"⚠️  Error generando respuesta {i+1}: {e}")
            respuestas.append(f"Error al generar respuesta {i+1}")
    
    return respuestas


In [ ]:
# Paso 3: Generar respuestas del modelo usando LLM
print("\n\n🤖 PASO 2: Generando respuestas del modelo con LLM...")
print("-" * 70)

num_respuestas_auto = 4
answers = await generar_respuestas_modelo(question, num_respuestas=num_respuestas_auto)

print(f"✓ Respuestas generadas: {len(answers)}")
for i, respuesta in enumerate(answers, 1):
    print(f"\n  Respuesta {i} ({len(respuesta)} caracteres):")
    print(f"  {respuesta[:150]}...")




🤖 PASO 2: Generando respuestas del modelo con LLM...
----------------------------------------------------------------------
✓ Respuestas generadas: 4

  Respuesta 1 (4389 caracteres):
  Respuesta clara y completa (usando únicamente la información del resumen):

Para maximizar simultáneamente la eficiencia relativa a Carnot y la robust...

  Respuesta 2 (1636 caracteres):
  Respuesta resumida y concreta:

Para maximizar simultáneamente la eficiencia relativa a Carnot y la robustez mecánica en un dispositivo termogalvánico...

  Respuesta 3 (7976 caracteres):
  Respuesta clara y técnica (basada únicamente en el resumen proporcionado)

Objetivo de diseño
- Maximizar simultáneamente la eficiencia relativa a Car...

  Respuesta 4 (1758 caracteres):
  Respuesta simple y directa:

Diseña el dispositivo como una célula termoeléctrica quasi-sólida (gel que confina el electrolito) y aplica simultáneamen...


In [35]:
print(respuestas_auto[2])

Respuesta clara y técnica (basada únicamente en el resumen proporcionado)

Objetivo de diseño
- Maximizar simultáneamente la eficiencia relativa a Carnot (es decir, obtener la mayor fracción posible de la energía térmica utilizable desde el gradiente térmico corporal) y la robustez mecánica (flexibilidad, resistencia al cizallamiento, durabilidad) de un dispositivo termogalvánico flexible, partiendo de las limitaciones y avances descritos en el resumen.

Principios físicos y de compromiso relevantes (extraídos del resumen)
- La eficiencia termogalvánica relativa a Carnot se incrementa al aumentar la diferencia de entropía de los pares redox (mayor termopolaridad/termopoder), al mantener alta conductividad iónica y cinética redox, y al reducir pérdidas térmicas y resistivas internas.
- Los dispositivos quasi‑sólidos (geles que confinan electrólitos) permiten flexibilidad pero presentan un trade‑off entre robustez mecánica y rendimiento termoelectroquímico: reforzar la matriz suele degra

### Generar rubricas adaptativas

In [ ]:
# ============================================================================
# FUNCIÓN PRINCIPAL: GENERAR RUBRICAS ADAPTATIVAS
# ============================================================================

from evolving_rubrics.prompts import get_adaptive_rubrics_prompt

async def generar_rubricas_adaptativas(
    pregunta: str,
    respuestas: List[str],
    rubricas_existentes: Optional[List[Dict[str, Any]]] = None) -> Optional[Dict[str, Any]]:
    # Construir el prompt usando la función del módulo prompts
    prompt_completo = get_adaptive_rubrics_prompt(pregunta, respuestas, rubricas_existentes)
    
    try:
        # Llamar al LLM
        respuesta_llm = await llamar_llm(
            prompt=prompt_completo,
            model=RUBRIC_GENERATION_MODEL
        )
        
        # Extraer JSON
        rubricas = extract_json_from_response(respuesta_llm)
        
        if rubricas:
            return rubricas
        else:
            print(f"⚠️  No se pudo extraer JSON de la respuesta del LLM")
            print(f"Respuesta recibida: {respuesta_llm[:200]}...")
            return None
            
    except Exception as e:
        print(f"❌ Error generando rubricas: {e}")
        return None

In [38]:
# ========================================================================
# PASO 3: Generar rubricas adaptativas
# ========================================================================
print("\n\n🧠 PASO 3: Generar rubricas adaptativas")
print("-" * 70)
print("Analizando diferencias entre respuestas...")

rubricas_adaptativas = await generar_rubricas_adaptativas(
    pregunta=question,
    respuestas=answers,
    rubricas_existentes=ground_truth_inicial["rubrics"]
)

if rubricas_adaptativas:
    print("\n✓ Rubricas adaptativas generadas:")
    
    if rubricas_adaptativas.get("positive_rubrics"):
        print(f"\n  Rubricas POSITIVAS ({len(rubricas_adaptativas['positive_rubrics'])}):")
        for i, rubric in enumerate(rubricas_adaptativas["positive_rubrics"], 1):
            print(f"    {i}. [{rubric.get('title', 'Sin título')}]")
            print(f"       {rubric['description'][:80]}...")
    
    if rubricas_adaptativas.get("negative_rubrics"):
        print(f"\n  Rubricas NEGATIVAS ({len(rubricas_adaptativas['negative_rubrics'])}):")
        for i, rubric in enumerate(rubricas_adaptativas["negative_rubrics"], 1):
            print(f"    {i}. [{rubric.get('title', 'Sin título')}]")
            print(f"       {rubric['description'][:80]}...")
else:
    print("❌ No se pudieron generar rubricas adaptativas")




🧠 PASO 3: Generar rubricas adaptativas
----------------------------------------------------------------------
Analizando diferencias entre respuestas...

✓ Rubricas adaptativas generadas:

  Rubricas POSITIVAS (2):
    1. [Mapeo función→diseño (mecanismo explícito)]
       Expone un mapeo explícito y verificable entre cada elección de diseño y la métri...
    2. [Plan de optimización cuantificable y validación]
       Propone un plan de optimización y validación accionable con parámetros clarament...

  Rubricas NEGATIVAS (1):
    1. [Afirmaciones causales contradictorias o no respaldadas]
       Afirmaciones causales activas o conclusiones que contradicen o no están respalda...


In [ ]:
rubricas_adaptativas['positive_rubrics']

[{'description': 'Expone un mapeo explícito y verificable entre cada elección de diseño y la métrica física que mejora: por ejemplo, identifica qué elemento (GdmCl, doble red, canales anisotrópicos, geometría de capas) afecta concretamente la termopower/Carnot-relative efficiency, cuál afecta la movilidad iónica/resistencia interna y cuál aporta rigidez/tenacidad mecánica. Una respuesta excelente no se limita a enumerar técnicas; indica para cada técnica el mecanismo físico (p. ej. aumento de diferencia de entropía, efecto Hofmeister, reducción de tortuosidad) y cómo mitiga el compromiso eficiencia–robustez descrito en el resumen.',
  'title': 'Mapeo función→diseño (mecanismo explícito)'},
 {'description': 'Propone un plan de optimización y validación accionable con parámetros claramente afinables y métricas medibles: especifica variables a ajustar (por ejemplo, concentración de GdmCl, grado de reticulación de cada red, orientación/tortuosidad de canales, espesor de capas), las magnitu

In [41]:
rubricas_adaptativas['negative_rubrics']

[{'description': 'Afirmaciones causales activas o conclusiones que contradicen o no están respaldadas por el resumen: por ejemplo, afirmar que un aditivo como GdmCl fortalece la red polimérica (cuando el resumen dice lo contrario), o asegurar la eliminación del trade‑off sin explicar mecanismos plausibles citados en el resumen. Se penaliza cualquier afirmación categórica sobre efectos materiales o resultados (p. ej. aumento de densidad de potencia) que vaya en contra de las limitaciones y compromisos explicitados en el resumen.',
  'title': 'Afirmaciones causales contradictorias o no respaldadas'}]

### Actualizar GT

In [ ]:
# ============================================================================
# FUNCIÓN: ACTUALIZAR GROUND TRUTH
# ============================================================================

def actualizar_ground_truth(
    ground_truth_inicial: Dict[str, Any],
    rubricas_adaptativas: Dict[str, Any]
) -> Dict[str, Any]:
    ground_truth_actualizado = ground_truth_inicial.copy()
    
    # Obtener rubricas persistentes originales
    rubricas_persistentes = ground_truth_inicial.get("rubrics", [])
    
    # Convertir rubricas adaptativas al formato correcto
    rubricas_nuevas = []
    
    # Rubricas positivas (peso +1.0)
    for rubric in rubricas_adaptativas.get("positive_rubrics", []):
        rubricas_nuevas.append({
            "description": rubric["description"],
            "weight": 1.0,
            "title": rubric.get("title", "Sin título")
        })
    
    # Rubricas negativas (peso -1.0)
    for rubric in rubricas_adaptativas.get("negative_rubrics", []):
        rubricas_nuevas.append({
            "description": rubric["description"],
            "weight": -1.0,
            "title": rubric.get("title", "Sin título")
        })
    
    # Combinar: persistentes primero, luego adaptativas
    ground_truth_actualizado["rubrics"] = rubricas_persistentes + rubricas_nuevas
    
    # Agregar tipos (opcional, para tracking)
    tipos = ["persistent"] * len(rubricas_persistentes) + ["adaptive"] * len(rubricas_nuevas)
    ground_truth_actualizado["rubrics_types"] = tipos
    
    return ground_truth_actualizado


In [43]:
# ========================================================================
# PASO 4: Actualizar Ground Truth
# ========================================================================
print("\n\n📝 PASO 4: Actualizar Ground Truth")
print("-" * 70)

ground_truth_final = actualizar_ground_truth(
    ground_truth_inicial,
    rubricas_adaptativas
)

print(f"Rubricas totales: {len(ground_truth_final['rubrics'])}")
print(f"  - Persistentes: {len(ground_truth_inicial['rubrics'])}")
print(f"  - Adaptativas: {len(ground_truth_final['rubrics']) - len(ground_truth_inicial['rubrics'])}")



📝 PASO 4: Actualizar Ground Truth
----------------------------------------------------------------------
Rubricas totales: 7
  - Persistentes: 4
  - Adaptativas: 3


In [44]:
ground_truth_final

{'query': "A continuación se muestra un resumen de contexto de un trabajo científico, seguido de una pregunta que debe responderse usando únicamente la información contenida en dicho resumen.\n\n---\nBackground Survey Summary\nPrior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:\nThermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to electricity, often faced limitations in efficiency, especially when dealing with low-grade heat sources like body heat. These devices typically suffered from low thermopower and were limited by rigid structures, which made them less suitable for wearable applications.\n2. Quasi-Solid Thermocells: The emergence of quasi-solid thermocells, which used gel networks to confine liquid elect

### Evaluar respuestas

In [45]:
# ============================================================================
# FUNCIÓN: EVALUAR RESPUESTA CON UNA RUBRICA
# ============================================================================

async def evaluar_rubrica(
    respuesta: str,
    pregunta: str,
    descripcion_rubrica: str
) -> float:
    """
    Evalúa una respuesta contra un criterio específico usando un LLM juez.
    
    Args:
        respuesta: La respuesta a evaluar
        pregunta: La pregunta original
        descripcion_rubrica: Descripción del criterio a evaluar
    
    Returns:
        Score entre 0.0 y 1.0
    """
    system_prompt = """You will be given a question someone asked (in <question></question> tags) and the corresponding response (in <response></response> tags) given to them by an assistant. You will then be given a specific criterion of the response to evaluate (in <criterion></criterion> tags).
Return a score on a scale of 0 to 2 indicating how appropriate the response is based on the given criterion. Judge only the specified aspect(s), not any other qualities of the answer. Output JSON in the format: {"score": x}."""
    
    user_prompt = f"""<question>{pregunta}</question>
<response>{respuesta}</response>
<criterion>{descripcion_rubrica}</criterion>"""
    
    try:
        respuesta_llm = await llamar_llm(
            prompt=user_prompt,
            system_prompt=system_prompt,
            model=RUBRIC_JUDGE_MODEL
        )
        
        resultado = extract_json_from_response(respuesta_llm)
        
        if resultado and "score" in resultado:
            score = float(resultado["score"])
            # Normalizar de 0-2 a 0-1
            return score / 2.0
        else:
            return 0.0
            
    except Exception as e:
        print(f"⚠️  Error evaluando rubrica: {e}")
        return 0.0


# ============================================================================
# FUNCIÓN: EVALUAR RESPUESTA CON TODAS LAS RUBRICAS
# ============================================================================

async def evaluar_respuesta_completa(
    respuesta: str,
    ground_truth: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Evalúa una respuesta contra todas las rubricas y calcula recompensa total.
    
    Args:
        respuesta: La respuesta a evaluar
        ground_truth: Ground truth con todas las rubricas
    
    Returns:
        Diccionario con scores por rubrica y recompensa total
    """
    pregunta = ground_truth["query"]
    rubricas = ground_truth["rubrics"]
    
    # Evaluar cada rubrica
    scores_por_rubrica = {}
    tareas = []
    
    for rubric in rubricas:
        descripcion = rubric["description"]
        # Crear clave única para la rubrica
        clave = rubric.get("title", descripcion[:30])
        tareas.append((clave, evaluar_rubrica(respuesta, pregunta, descripcion)))
    
    # Ejecutar todas las evaluaciones en paralelo
    resultados = await asyncio.gather(*[tarea[1] for tarea in tareas])
    
    for (clave, _), score in zip(tareas, resultados):
        scores_por_rubrica[clave] = score
    
    # Calcular recompensa total (promedio ponderado)
    recompensa_total = 0.0
    peso_total = 0.0
    
    for i, rubric in enumerate(rubricas):
        clave = rubric.get("title", rubric["description"][:30])
        peso = abs(rubric["weight"])  # Usar valor absoluto para el peso
        score = scores_por_rubrica[clave]
        
        # Multiplicar por el signo del weight (positivo o negativo)
        recompensa_total += score * rubric["weight"] * peso
        peso_total += peso
    
    recompensa_final = recompensa_total / peso_total if peso_total > 0 else 0.0
    
    return {
        "recompensa_total": recompensa_final,
        "scores_por_rubrica": scores_por_rubrica,
        "num_rubricas": len(rubricas)
    }

In [ ]:
# ========================================================================
# PASO 5: Evaluar respuestas
# ========================================================================
print("\n\n📊 PASO 5: Evaluar respuestas con todas las rubricas")
print("-" * 70)

evaluaciones = []
for i, respuesta in enumerate(answers, 1):
    print(f"\n  Evaluando Respuesta {i}...")
    resultado = await evaluar_respuesta_completa(respuesta, ground_truth_final)
    evaluaciones.append({
        'respuesta_num': i,
        **resultado
    })
    print(f"    Recompensa total: {resultado['recompensa_total']:.3f}")



📊 PASO 5: Evaluar respuestas con todas las rubricas
----------------------------------------------------------------------

  Evaluando Respuesta 1...
    Recompensa total: 0.643

  Evaluando Respuesta 2...
    Recompensa total: 0.643

  Evaluando Respuesta 3...
    Recompensa total: 0.643

  Evaluando Respuesta 4...
    Recompensa total: 0.714


### Resumen

In [48]:
# ========================================================================
# RESUMEN
# ========================================================================
print("\n\n" + "="*70)
print("📈 RESUMEN FINAL")
print("="*70)

print(f"\nRubricas:")
print(f"  - Persistentes: {len(ground_truth_inicial['rubrics'])}")
print(f"  - Adaptativas: {len(ground_truth_final['rubrics']) - len(ground_truth_inicial['rubrics'])}")
print(f"  - Total: {len(ground_truth_final['rubrics'])}")

print(f"\nRecompensas por respuesta:")
for eval_result in evaluaciones:
    print(f"  Respuesta {eval_result['respuesta_num']}: {eval_result['recompensa_total']:.3f}")

mejor = max(evaluaciones, key=lambda x: x['recompensa_total'])
peor = min(evaluaciones, key=lambda x: x['recompensa_total'])

print(f"\n✓ Mejor respuesta: Respuesta {mejor['respuesta_num']} (recompensa: {mejor['recompensa_total']:.3f})")
print(f"✗ Peor respuesta: Respuesta {peor['respuesta_num']} (recompensa: {peor['recompensa_total']:.3f})")

print("\n" + "="*70)
print("✓ Proceso completado!")
print("="*70)

output = {
    'ground_truth_inicial': ground_truth_inicial,
    'rubricas_adaptativas': rubricas_adaptativas,
    'ground_truth_final': ground_truth_final,
    'evaluaciones': evaluaciones
    }



📈 RESUMEN FINAL

Rubricas:
  - Persistentes: 4
  - Adaptativas: 3
  - Total: 7

Recompensas por respuesta:
  Respuesta 1: 0.643
  Respuesta 2: 0.643
  Respuesta 3: 0.643
  Respuesta 4: 0.714

✓ Mejor respuesta: Respuesta 4 (recompensa: 0.714)
✗ Peor respuesta: Respuesta 1 (recompensa: 0.643)

✓ Proceso completado!


In [49]:
output

{'ground_truth_inicial': {'query': "A continuación se muestra un resumen de contexto de un trabajo científico, seguido de una pregunta que debe responderse usando únicamente la información contenida en dicho resumen.\n\n---\nBackground Survey Summary\nPrior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:\nThermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to electricity, often faced limitations in efficiency, especially when dealing with low-grade heat sources like body heat. These devices typically suffered from low thermopower and were limited by rigid structures, which made them less suitable for wearable applications.\n2. Quasi-Solid Thermocells: The emergence of quasi-solid thermocells, which used gel network

In [51]:
evaluaciones

[{'respuesta_num': 1,
  'recompensa_total': 0.6428571428571429,
  'scores_por_rubrica': {'Adherencia al resumen': 1.0,
   'Análisis del compromiso eficiencia–robustez': 1.0,
   'Propuesta de diseño coherente con objetivos': 1.0,
   'Claridad y lógica de la argumentación': 1.0,
   'Mapeo función→diseño (mecanismo explícito)': 1.0,
   'Plan de optimización cuantificable y validación': 0.5,
   'Afirmaciones causales contradictorias o no respaldadas': 1.0},
  'num_rubricas': 7},
 {'respuesta_num': 2,
  'recompensa_total': 0.6428571428571429,
  'scores_por_rubrica': {'Adherencia al resumen': 0.5,
   'Análisis del compromiso eficiencia–robustez': 1.0,
   'Propuesta de diseño coherente con objetivos': 1.0,
   'Claridad y lógica de la argumentación': 1.0,
   'Mapeo función→diseño (mecanismo explícito)': 1.0,
   'Plan de optimización cuantificable y validación': 0.5,
   'Afirmaciones causales contradictorias o no respaldadas': 0.5},
  'num_rubricas': 7},
 {'respuesta_num': 3,
  'recompensa_tota

# Datasets

## ResearchBench

In [3]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("ankilok/Researchbench")

/Users/lucaspecina/miniforge3/envs/RL/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 678/678 [00:00<00:00, 16487.92 examples/s]


In [6]:
df = ds["train"].to_pandas()
df

,Title,Background Little Survey,Background Question,Main Inspiration,Inspiration paper 1 title,Relation between the main inspiration and the inspiration paper 1,Inspiration paper 2 title,Relation between the main inspiration and the inspiration paper 2,Inspiration paper 3 title,Relation between the main inspiration and the inspiration paper 3,Main hypothesis,doi,abstract,experiments details,fine grained hypothesis
0,A small and vigorous black hole in the early U...,### Background Survey\n\n1. **Theoretical Mode...,How can we detect and characterize black holes...,None,The Near-Infrared Spectrograph (NIRSpec) on th...,"Yes, this paper can certainly serve as an insp...",The assembly of the first massive black holes,"Yes, this paper can certainly serve as an insp...",Nuclear activity versus star formation: emissi...,"Yes, this paper can certainly serve as an insp...","The galaxy GN-z11, observed at a redshift of 1...",10.1038/s41586-024-07052-5,Several theories have been proposed to describ...,Thank you for your patience and for prompting ...,The main scientific finding of this paper is t...
1,Most of the photons that reionized the Univers...,Certainly! Here’s a more concise version of th...,What are the sources that drove cosmic reioniz...,None,Are ultra-faint galaxies at z = 6–8 responsibl...,"Yes, this paper can serve as an inspiration fo...",The Lyman-continuum photon production efficien...,"Yes, this paper can indeed serve as an inspira...",Early galaxy formation and its large-scale eff...,"Yes, this paper can indeed serve as an inspira...","The paper posits that faint galaxies, particul...",10.1038/s41586-024-07043-6,The identification of sources driving cosmic r...,Certainly! Below is the revised summary of exp...,The main scientific finding of this paper is t...
2,A high black-hole-to-host mass ratio in a lens...,To summarize the background knowledge pertinen...,How does the early growth phase of supermassiv...,None,UNCOVER: candidate red active galactic nuclei ...,"Based on the provided abstract and context, it...",UNCOVERing the extended strong lensing structu...,"Based on the abstract provided, the paper titl...",None,None,The paper posits that the object Abell2744-QSO...,10.1038/s41586-024-07184-8,Early JWST observations have uncovered a popul...,Thank you for your patience. Upon reviewing th...,The main scientific finding of this paper cent...
3,Bound star clusters observed in a lensed galax...,To summarize the background knowledge pertinen...,How can observations from the James Webb Space...,None,Performance of NIRCam on JWST in Flight,Based on the abstract provided for the paper t...,Most of the photons that reionized the Univers...,Based on the abstract provided for the paper t...,None,None,"The Cosmic Gems arc, observed using JWST, cons...",10.1038/s41586-024-07703-7,The Cosmic Gems arc is among the brightest and...,Thank you for your patience. After reviewing t...,The main scientific finding of this paper is t...
4,A radius valley between migrated steam worlds ...,### Background Survey\n\n1. **Exoplanet Census...,How can we explain the observed radius valley ...,None,The California- Kepler Survey. III. A gap in t...,"Yes, the paper titled ""The California-Kepler S...",The nature of the radius valley,"Yes, the paper titled ""The nature of the radiu...",None,None,The observed radius valley in the distribution...,10.1038/s41550-023-02183-7,The radius valley (or gap) in the observed dis...,Certainly! Here’s the revised summary that inc...,The main scientific finding of this paper is t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,Universal paradigm of ternary metacomposites w...,Certainly! Here’s a more concise version of th...,How can tunable weakly negative permittivity a...,None,Precise regulation of weakly negative permitti...,"Based on the information provided, the paper t...",Tunable radio-frequency negative permittivity ...,"Based on the provided abstract and analysis, t...",Radio-fre

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 15 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   Title                                                              678 non-null    object
 1   Background Little Survey                                           678 non-null    object
 2   Background Question                                                678 non-null    object
 3   Main Inspiration                                                   0 non-null      object
 4   Inspiration paper 1 title                                          678 non-null    object
 5   Relation between the main inspiration and the inspiration paper 1  678 non-null    object
 6   Inspiration paper 2 title                                          678 non-null    object
 7   Relation between the main inspirati

In [ ]:
print(df.loc[4, 'Title']) # https://arxiv.org/abs/2401.04380

A radius valley between migrated steam worlds and evaporated rocky cores


In [ ]:
row = df.loc[4] # https://arxiv.org/abs/2401.04380
for col in df.columns:
    print(f"{col}:\n{row[col]}\n{'-'*100}")

Title:
A radius valley between migrated steam worlds and evaporated rocky cores
--------------------------------------------------------------------------------------------------------
Background Little Survey:
### Background Survey

1. **Exoplanet Census and Distribution**: Prior to this study, extensive observational campaigns, notably through missions such as Kepler, cataloged a significant number of exoplanets, revealing a pronounced gap in the radius distribution known as the radius valley, which separates smaller super-Earths from larger sub-Neptunes. This phenomenon prompted inquiries into the underlying mechanisms responsible for such a distribution.

2. **Core Accretion Model**: The core accretion model has been fundamental in understanding planet formation, positing that planets form through the gradual accumulation of solid materials within a protoplanetary disk. It emphasizes the role of local conditions, such as temperature gradients and the presence of ice lines, which in

CAMPOS:
- background little survey: 

## RaR Science

In [2]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
splits = {'train': 'data/train-00000-of-00001.parquet', 'val': 'data/val-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/anisha2102/RaR-Science-20k-o3-mini/" + splits["train"])

/Users/lucaspecina/miniforge3/envs/RL/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df

,question,reference_answer,question_source,rubric,rubric_list,rubric_count
0,Values of \( K_p \) are obtained at a variety ...,-44.9 kcal,INFLYTECH/SCP-116K,[{'description': 'Essential Criteria: The resp...,[Essential Criteria: The response should menti...,7
1,How many visible photons (∼ 5000 Å) does a 100...,10^{19},General/VNet,[{'description': 'Essential Criteria: The resp...,[Essential Criteria: The response must explici...,8
2,Consider a dumbbell-shaped artificial satellit...,The rotating reference frame with the geocente...,INFLYTECH/SCP-116K,[{'description': 'Essential Criteria: The resp...,[Essential Criteria: The response must clearly...,8
3,Consider a 2D Ising model on a square lattice ...,"Second-order phase transition for B=0, and fir...",Meta/natural_reasoning,[{'description': 'Essential Criteria: The resp...,[Essential Criteria: The response must correct...,7
4,A hydrogen atom is placed in a magnetic field....,The solutions with the same n and l but differ...,Meta/natural_reasoning,[{'description': 'Essential Criteria: Checks i...,[Essential Criteria: Checks if the response ex...,7
...,...,...,...,...,...,...
18328,What is the fundamental nature of uncertainty ...,The uncertainty principle suggests that activi...,Meta/natural_reasoning,[{'description': 'Essential Criteria: Clearly ...,[Essential Criteria: Clearly explains that the...,7
18329,Given the equations and percent yield for the ...,31.8 kg,Meta/natural_reasoning,[{'description': 'Essential Criteria: The resp...,[Essential Criteria: The response must correct...,7
18330,Camphor has been found to undergo a crystallin...,3.09 × 10^9 N/m^2 = (3.09 × 10^9 Pa)(1 atm/101...,INFLYTECH/SCP-116K,[{'description': 'Essential Criteria: The resp...,[Essential Criteria: The response must correct...,7
18331,Derive the expression for angular momentum usi...,L_{\\mu\\nu} = x_\\mu p_\\nu - x_\\nu p_\\mu,Meta/natural_reasoning,[{'description': 'Essential Criteria: Provides...,[Essential Criteria: Provides a correct deriva...,7


In [4]:
df.question_source.value_counts()

question_source
Meta/natural_reasoning    9241
INFLYTECH/SCP-116K        7072
General/VNet              2020
Name: count, dtype: int64

In [ ]:
row = df.loc[4]
for col in df.columns:
    print(f"{col}:\n{row[col]}\n{'-'*100}")

question:
A hydrogen atom is placed in a magnetic field. The Schrödinger equation is solved to obtain the probability distribution of the atom's orientation. Explain why the solutions with the same n and l but different m values are not spherically symmetric, and how the application of a magnetic field breaks the spherical symmetry. Provide a detailed analysis of the energy levels of the atom in the presence of the magnetic field, including the alignment of the z-axis along the magnetic field direction.
----------------------------------------------------------------------------------------------------
reference_answer:
The solutions with the same n and l but different m values are not spherically symmetric because they correspond to different orientations of the atom's angular momentum. The application of a magnetic field breaks the spherical symmetry by introducing a preferred direction, which is the direction of the magnetic field. The energy levels of the atom in the presence of th

In [6]:
row = df.loc[100]
for col in df.columns:
    print(f"{col}:\n{row[col]}\n{'-'*100}")

question:
PKU is an inborn error of metabolism of the amino acid phenylalanine. The following pedigree is of an affected family.

Pedigree Description:
- Generation I: One male, one female, unaffected.
- Generation II: Two parents, one affected male, one affected female, two unaffected (one male, one female).
- Generation III: One affected female, four unaffected (three males, one female).

a. What is the mode of inheritance of PKU?
b. Which persons are known to be heterozygous for PKU?
c. What is the probability that III-2 is heterozygous?
d. If III-3 and III-4 marry, what is the probability that their first child will have PKU?
----------------------------------------------------------------------------------------------------
reference_answer:
a. autosomal recessive.

b. I-1, I-2, II-1, II-2, III-4, III-5

c. \( \frac{2}{3} \)

d. \( \frac{1}{6} \)

Since the normal individuals have an affected child, the trait must be recessive (concept 6-B). It can’t be X-linked recessive; I-1 wou

## OpenRubrics

In [7]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_json("hf://datasets/OpenRubrics/OpenRubrics/data/open_rubrics.jsonl", lines=True)
df

,instruction,rubric,source,judge,response_a,response_b,winner
0,Can you write a C++ program that prompts the u...,1. The response includes code that prompts the...,ultrafeedback,--- Compliance Check ---\nIdentified Gatekeepe...,"Sure, here is the program using the C++11 algo...",Here's a C++ program that prompts the user to ...,B
1,Write a program in Python that generates a lis...,1. The response fulfills all explicit instruct...,ultrafeedback,--- Compliance Check ---\nIdentified Gatekeepe...,"To achieve this, we would need to use a combin...",The person in the image appears to be feeling ...,A
2,Can you determine the missing number in the se...,1. The response identifies and applies the exp...,ultrafeedback,--- Compliance Check ---\nIdentified Gatekeepe...,Of course! Let's analyze the sequence more clo...,"Sure, I'd be happy to help you with that!\n\nT...",A
3,How can I use SQL to extract data related to t...,1. The response includes a clear and relevant ...,ultrafeedback,--- Compliance Check ---\nIdentified Gatekeepe...,"Certainly! As an AI assistant, I'm happy to he...",SELECT * FROM Multinational_Corporations WHERE...,A
4,Create a table in Excel to classify the follow...,1. The response creates a table that clearly a...,ultrafeedback,--- Compliance Check ---\nIdentified Gatekeepe...,Hello! I'm here to help! 😊\n\nTo classify the ...,"Sure, here is a table in Excel that classifies...",A
...,...,...,...,...,...,...,...
35617,after i got high fever for 3 days 2 and half m...,1. The response addresses all explicit element...,health_medicine,--- Compliance Check ---\nIdentified Gatekeepe...,Thank you for sharing your detailed history. W...,It is important to consult with a doctor or de...,A
35618,Hi Dr Last January I was in the Hospital for ...,1. The response must address all explicit ques...,health_medicine,--- Compliance Check ---\nIdentified Gatekeepe...,"Okay, this is a *very* understandably frighten...",I can't answer that for you. If you are concer...,A
35619,My sister had a anueryism burst last week and ...,1. The response must address the explicit ques...,health_medicine,--- Compliance Check ---\nIdentified Gatekeepe...,I can't answer that for you. If you are concer...,I'm deeply sorry to hear about your sister’s c...,B
35620,Hi. I am diagnosed with PCOS when I was 17. No...,1. The response must address all explicit ques...,health_medicine,--- Compliance Check ---\nIdentified Gatekeepe...,I can't provide a medical diagnosis. If you ar...,"Dear Aira,\n\nThank you for sharing your detai...",B


In [14]:
science_sources = [
  "biology", "chemistry", "health_medicine",
  "medical_o1", "medicine", "physics"
]

df = df[df.source.isin(science_sources)].reset_index(drop=True)
df

,instruction,rubric,source,judge,response_a,response_b,winner
0,Given the symptoms of sudden weakness in the l...,1. The response directly addresses the specifi...,medical_o1,--- Compliance Check ---\nIdentified Gatekeepe...,The specific cardiac abnormality most likely t...,Deep vein thrombosis (DVT) with potential pulm...,A
1,A 33-year-old woman is brought to the emergenc...,1. The response directly addresses the specifi...,medical_o1,--- Compliance Check ---\nIdentified Gatekeepe...,The diaphragm is likely injured due to its pro...,"In this scenario, the most likely anatomical s...",B
2,A 45-year-old man with a history of alcohol us...,1. The response directly addresses the diagnos...,medical_o1,--- Compliance Check ---\nIdentified Gatekeepe...,Wernicke's encephalopathy.,**Cerebellar Degeneration (likely alcohol-rela...,B
3,A patient with psoriasis was treated with syst...,1. The response directly addresses the cause o...,medical_o1,--- Compliance Check ---\nIdentified Gatekeepe...,The development of generalized pustules in a p...,The most likely cause of this condition is a p...,A
4,Scientists are developing a new non-steroidal ...,1. The response must address all explicit cond...,medical_o1,--- Compliance Check ---\nIdentified Gatekeepe...,To determine which curve would represent the n...,Curve B,A
...,...,...,...,...,...,...,...
11344,after i got high fever for 3 days 2 and half m...,1. The response addresses all explicit element...,health_medicine,--- Compliance Check ---\nIdentified Gatekeepe...,Thank you for sharing your detailed history. W...,It is important to consult with a doctor or de...,A
11345,Hi Dr Last January I was in the Hospital for ...,1. The response must address all explicit ques...,health_medicine,--- Compliance Check ---\nIdentified Gatekeepe...,"Okay, this is a *very* understandably frighten...",I can't answer that for you. If you are concer...,A
11346,My sister had a anueryism burst last week and ...,1. The response must address the explicit ques...,health_medicine,--- Compliance Check ---\nIdentified Gatekeepe...,I can't answer that for you. If you are concer...,I'm deeply sorry to hear about your sister’s c...,B
11347,Hi. I am diagnosed with PCOS when I was 17. No...,1. The response must address all explicit ques...,health_medicine,--- Compliance Check ---\nIdentified Gatekeepe...,I can't provide a medical diagnosis. If you ar...,"Dear Aira,\n\nThank you for sharing your detai...",B


In [17]:
row = df[df.source == 'biology'].reset_index(drop=True).loc[10]
for col in df.columns:
    print(f"{col}:\n{row[col]}\n{'-'*100}")

instruction:
In the vicinity of a nuclear reprocessing plant, 4 cases of childhood leukaemia were observed over a certain period, while national registration rates would have expected only 0.25 cases. Compare the log likelihood ratio test and the score test for the null hypothesis that the incidence rates of leukaemia in the area do not differ from the national rates.
----------------------------------------------------------------------------------------------------
rubric:
1. The response correctly addresses all explicit commands and constraints stated in the request, including comparing the two specified statistical tests for the given null hypothesis. [Hard Rule] 2. The response clearly defines the null and alternative hypotheses in a manner consistent with the request. [Hard Rule] 3. The response explicitly computes or discusses the relevant test statistics for both the log-likelihood ratio test and the score test. [Hard Rule] 4. The response uses appropriate statistical models an

## DR-Tulu

Creo que es sacado de RaR

In [18]:
import pandas as pd

df = pd.read_parquet("hf://datasets/rl-research/dr-tulu-rl-data/data/train-00000-of-00001.parquet")
df 

,source,question_type,messages,ground_truth,dataset
0,rl_rag_train_sqa_1k_clean_search_rubric_longfo...,long_form,[{'content': 'What is the reactivity constant ...,"{""query"": ""What is the reactivity constant of ...",general_rubric
1,rl_rag_train_sqa_1k_clean_search_rubric_longfo...,long_form,[{'content': 'During electrochemical etching o...,"{""query"": ""During electrochemical etching of F...",general_rubric
2,rl_rag_train_sqa_1k_clean_search_rubric_longfo...,long_form,[{'content': '34. To what extent are students ...,"{""query"": ""34.\tTo what extent are students in...",general_rubric
3,rl_rag_train_sqa_1k_clean_search_rubric_longfo...,long_form,[{'content': 'for IV TPN and vitamin bag prepa...,"{""query"": ""for IV TPN and vitamin bag preparat...",general_rubric
4,rl_rag_train_sqa_1k_clean_search_rubric_longfo...,long_form,[{'content': 'Instructions: You are expected t...,"{""query"": ""Instructions: You are expected to e...",general_rubric
...,...,...,...,...,...
4876,rl_rag_train_sa_3k_longform_rubrics_adaptive_r...,long_form,[{'content': 'can I use my time in boot camp a...,"{""query"": ""can I use my time in boot camp as p...",general_rubric
4877,rl_rag_train_sa_3k_longform_rubrics_adaptive_r...,long_form,[{'content': 'Summarize the notable privacy-re...,"{""query"": ""Summarize the notable privacy-relat...",general_rubric
4878,rl_rag_train_sa_3k_longform_rubrics_adaptive_r...,long_form,"[{'content': 'cisplatin inhibit p-p65', 'role'...","{""query"": ""cisplatin inhibit p-p65"", ""rubrics""...",general_rubric
4879,rl_rag_train_sa_3k_longform_rubrics_adaptive_r...,long_form,[{'content': 'Summarize the notable privacy-re...,"{""query"": ""Summarize the notable privacy-relat...",general_rubric


In [20]:
row = df.loc[10]
for col in df.columns:
    print(f"{col}:\n{row[col]}\n{'-'*100}")

source:
rl_rag_train_sqa_1k_clean_search_rubric_longform_rubrics_adaptive_rubric
----------------------------------------------------------------------------------------------------
question_type:
long_form
----------------------------------------------------------------------------------------------------
messages:
[{'content': 'do a literature review on cable anchor and also their potential to replace timber support in hardrock mines', 'role': 'user'}]
----------------------------------------------------------------------------------------------------
ground_truth:
{"query": "do a literature review on cable anchor and also their potential to replace timber support in hardrock mines", "rubrics": [{"description": "Define cable anchors, their basic purpose, and their core mechanical principles in ground support.", "title": "Describe cable anchors and core mechanics", "weight": 3}, {"description": "Explain the use of cable anchors/bolts in hardrock mines, including current deployment, pr

In [25]:
row['messages'][0]

{'content': 'do a literature review on cable anchor and also their potential to replace timber support in hardrock mines',
 'role': 'user'}

In [29]:
import json
json.loads(row['ground_truth'])

{'query': 'do a literature review on cable anchor and also their potential to replace timber support in hardrock mines',
 'rubrics': [{'description': 'Define cable anchors, their basic purpose, and their core mechanical principles in ground support.',
   'title': 'Describe cable anchors and core mechanics',
   'weight': 3},
  {'description': 'Explain the use of cable anchors/bolts in hardrock mines, including current deployment, prevalence, and examples.',
   'title': 'Discuss cable anchor use in hardrock mining',
   'weight': 3},
  {'description': 'Compare the mechanical properties and support capabilities of cable anchors/bolts versus timber supports in hardrock environments.',
   'title': 'Compare cable anchors to timber supports',
   'weight': 3},
  {'description': 'Evaluate the potential for cable anchors to replace timber supports in hardrock mines, specifically addressing performance, adaptability, and current research directions.',
   'title': 'Assess replacement of timber supp

## MOOSE-Chem

In [7]:
import pandas as pd

df = pd.read_excel('data/moose-chem/chem_research_2024.xlsx')
df.head()

,No,Title,Public Date,Publisher,Background Little Survey,Background Little Survey (strict),Background Question,Background Question (strict),Main Inspiration,Inspiration paper 1 title,Relation between the main inspiration and the inspiration paper 1,Inspiration paper 2 title,Relation between the main inspiration and the inspiration paper 2,Inspiration paper 3 title,Relation between the main inspiration and the inspiration paper 3,Main hypothesis,Experiments to Verify the Research Hypothesis:,Reasoning Process,Note
0,0,"Ultrastrong, flexible thermogalvanic armor wit...",2024/08/07,Nature Communication,Background Survey Summary\nPrior to the develo...,Background Survey Summary\nPrior to the develo...,How can we design a flexible thermogalvanic de...,NaN,The main inspiration for the paper stems from ...,Interactions between macromolecules and ions: ...,Their work on the Hofmeister series (cited as ...,Beyond Hofmeister,Insights into using the Hofmeister series to m...,Strong tough hydrogels via the synergy of free...,This paper demonstrates the synergy of freeze-...,By integrating guanidine sulfate (Gdm)2SO4 int...,Thermoelectric Performance Tests: The FTGA exh...,bkg + insp1/insp2 + insp3 = hyp,insp1/2: The Hofmeister series; insp3: directi...
1,1,Chaotropic Effect-Boosted Thermogalvanic Ionog...,2024/01/29,Advanced Materials,Thermoelectric Energy Conversion: Traditional ...,Thermoelectric Energy Conversion: Traditional ...,How can a thermoelectric system be designed to...,How can a thermoelectric system be designed to...,The main inspiration of the paper comes from t...,The Chaotropic Effect as an Assembly Motif in ...,"Have the keyword ""Chaotropic Effect""",Role of Ions in Hydrogels with an Ionic Seebec...,This paper explains how the Soret effect and s...,Hierarchically porous polymer coatings for hig...,This paper details the creation of radiative c...,The hypothesis of the paper is that integratin...,The experiments included the design and testin...,bkg + insp1 + insp2 + insp3 = hyp,insp1: Chaotropic Effect (ref id: find online ...
2,2,Synergistic Anisotropic Network and Hierarchic...,2024/02/01,Small,Background survey:\n\nHarnessing Low-Grade Hea...,NaN,How can a cost-effective N-type quasi-solid-st...,NaN,The main inspiration of this work is the syner...,3D Hierarchical Electrodes Boosting Ultrahigh ...,It directly inspires the design of the hierarc...,Cu(ii/i) redox couples: potential alternatives...,This work supports the use of copper as the re...,Anti-Fatigue and Highly Conductive Thermocells...,It directly inspired the structural design of ...,The hypothesis is that combining an anisotropi...,The experiments designed to test the hypothesi...,bkg + insp1 + insp2 + insp3 = hyp,insp1: Hierarchical 3D Copper Electrodes (find...
3,3,Liquid-flow thermocells with high hybrid entro...,2024/07/14,Nano Energy,Background Survey\nThermoelectric Conversion f...,NaN,How can the thermoelectric performance of liqu...,How can the thermoelectric performance of liqu...,The main inspiration of this paper is the hybr...,Effect of solvation shell structure on thermop...,It demonstrated theThis paper serves as a foun...,High thermopower of ferri/ferrocyanide redox c...,This paper introduces the concept of using a c...,NaN,NaN,The hypothesis is that inducing hybrid entropy...,The experiments include fabricating LFTCs with...,bkg + insp1 + insp2 = hyp,insp1: Hybrid Entropy Increase through Flow-Se...
4,4,Tough and elastic hydrogel thermocells for hea...,2024/08/01,Chemical Engineering Journal,Background Survey:\nHydrogel thermocells have ...,NaN,How can a hydrogel thermocell be designed to a...,How can a hydrogel thermocell be designed to a...,The main inspiration behind this paper is a bi...,"Fracture, fatigue, and friction of polymers in...",This paper focuses on the role of dense polyme...,Highly stretchable and tough hydrogels,"This paper shows how a dual-network approach, ...",A review of radiation-grafted polymer electrol...,increased m

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 19 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   No                                                                 51 non-null     int64 
 1   Title                                                              51 non-null     object
 2   Public Date                                                        51 non-null     object
 3   Publisher                                                          51 non-null     object
 4   Background Little Survey                                           51 non-null     object
 5   Background Little Survey (strict)                                  8 non-null      object
 6   Background Question                                                51 non-null     object
 7   Background Question (strict)         

In [8]:
df.head().to_csv('data/moose-chem/chem_research_2024-sample.csv', index=False)

In [34]:
row = df.loc[0]
for col in df.columns:
    print(f"{col}:\n{row[col]}\n{'-'*100}")

No:
0
----------------------------------------------------------------------------------------------------
Title:
Ultrastrong, flexible thermogalvanic armor with a Carnot-relative efficiency over 8%
----------------------------------------------------------------------------------------------------
Public Date:
2024/08/07
----------------------------------------------------------------------------------------------------
Publisher:
Nature Communication
----------------------------------------------------------------------------------------------------
Background Little Survey:
Background Survey Summary
Prior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:
Thermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to elect

## MOOSE-Chem2 (fine-grained)

In [53]:
import pandas as pd

df = pd.read_excel('data/moose-chem2/chem_research_2024_finegrained.xlsx')
df.head()

,No,Title,Public Date,Publisher,Background Little Survey,Background Little Survey (strict),Background Question,Background Question (strict),Main Inspiration,Inspiration paper 1 title,...,Inspiration paper 2 title,Relation between the main inspiration and the inspiration paper 2,Inspiration paper 3 title,Relation between the main inspiration and the inspiration paper 3,Main hypothesis,Experiments to Verify the Research Hypothesis:,Reasoning Process,Note,Finegrained Hypothesis,Finegrained Experiment
0,0,"Ultrastrong, flexible thermogalvanic armor wit...",2024/08/07,Nature Communication,Background Survey Summary\nPrior to the develo...,Background Survey Summary\nPrior to the develo...,How can we design a flexible thermogalvanic de...,NaN,The main inspiration for the paper stems from ...,Interactions between macromolecules and ions: ...,...,Beyond Hofmeister,Insights into using the Hofmeister series to m...,Strong tough hydrogels via the synergy of free...,This paper demonstrates the synergy of freeze-...,By integrating guanidine sulfate (Gdm)2SO4 int...,Thermoelectric Performance Tests: The FTGA exh...,bkg + insp1/insp2 + insp3 = hyp,insp1/2: The Hofmeister series; insp3: directi...,The paper presents a methodology for creating ...,### **1. Thermopower Measurement**\n\n- **Obje...
1,1,Chaotropic Effect-Boosted Thermogalvanic Ionog...,2024/01/29,Advanced Materials,Thermoelectric Energy Conversion: Traditional ...,Thermoelectric Energy Conversion: Traditional ...,How can a thermoelectric system be designed to...,How can a thermoelectric system be designed to...,The main inspiration of the paper comes from t...,The Chaotropic Effect as an Assembly Motif in ...,...,Role of Ions in Hydrogels with an Ionic Seebec...,This paper explains how the Soret effect and s...,Hierarchically porous polymer coatings for hig...,This paper details the creation of radiative c...,The hypothesis of the paper is that integratin...,The experiments included the design and testin...,bkg + insp1 + insp2 + insp3 = hyp,insp1: Chaotropic Effect (ref id: find online ...,### Scientific Finding with Full Details\n\nTh...,### **1. Material Design and Synthesis**\n\n##...
2,2,Synergistic Anisotropic Network and Hierarchic...,2024/02/01,Small,Background survey:\n\nHarnessing Low-Grade Hea...,NaN,How can a cost-effective N-type quasi-solid-st...,NaN,The main inspiration of this work is the syner...,3D Hierarchical Electrodes Boosting Ultrahigh ...,...,Cu(ii/i) redox couples: potential alternatives...,This work supports the use of copper as the re...,Anti-Fatigue and Highly Conductive Thermocells...,It directly inspired the structural design of ...,The hypothesis is that combining an anisotropi...,The experiments designed to test the hypothesi...,bkg + insp1 + insp2 + insp3 = hyp,insp1: Hierarchical 3D Copper Electrodes (find...,### **Scientific Finding**\n\n#### **Problem A...,### **Review Methodology**\n1. **Check for Mis...
3,3,Liquid-flow thermocells with high hybrid entro...,2024/07/14,Nano Energy,Background Survey\nThermoelectric Conversion f...,NaN,How can the thermoelectric performance of liqu...,How can the thermoelectric performance of liqu...,The main inspiration of this paper is the hybr...,Effect of solvation shell structure on thermop...,...,High thermopower of ferri/ferrocyanide redox c...,This paper introduces the concept of using a c...,NaN,NaN,The hypothesis is that inducing hybrid entropy...,The experiments include fabricating LFTCs with...,bkg + insp1 + insp2 = hyp,insp1: Hybrid Entropy Increase through Flow-Se...,### Methodology Design\n1. **Liquid-flow Therm...,1. **Fabrication and Design of LFTCs**:\n - ...
4,4,Tough and elastic hydrogel thermocells for hea...,2024/08/01,Chemical Engineering Journal,Background Survey:\nHydrogel thermocells have ...,NaN,How can a hydrogel thermocell be designed to a...,How can a hydrogel thermocell be designed to a...,The main inspiration behind this paper is a bi...,"Fracture, fatigue, and friction of polymers in

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 21 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   No                                                                 51 non-null     int64 
 1   Title                                                              51 non-null     object
 2   Public Date                                                        51 non-null     object
 3   Publisher                                                          51 non-null     object
 4   Background Little Survey                                           51 non-null     object
 5   Background Little Survey (strict)                                  8 non-null      object
 6   Background Question                                                51 non-null     object
 7   Background Question (strict)         

In [ ]:
# df.head().to_csv('moose-chem/chem_research_2024-sample.csv', index=False)

In [ ]:
row = df.loc[0]
for col in df.columns:
    print(f"{col}:\n{row[col]}\n{'-'*100}")

No:
0
----------------------------------------------------------------------------------------------------
Title:
Ultrastrong, flexible thermogalvanic armor with a Carnot-relative efficiency over 8%
----------------------------------------------------------------------------------------------------
Public Date:
2024/08/07
----------------------------------------------------------------------------------------------------
Publisher:
Nature Communication
----------------------------------------------------------------------------------------------------
Background Little Survey:
Background Survey Summary
Prior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:
Thermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to elect